In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import tweepy
import re 
from tweepy import OAuthHandler 
from textblob import TextBlob
import requests
from bs4 import BeautifulSoup


In [2]:
def get_top_five():
    #Parsing del link
    url = 'https://www.imdb.com/chart/tvmeter?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=4da9d9a5-d299-43f2-9c53-f0efa18182cd&pf_rd_r=0KYJP3BJH5DN6EZWRQ5Z&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=toptv&ref_=chttvtp_ql_5'
    soup = requests.get(url).content
    soup = BeautifulSoup(soup,'html.parser')
    
    #Conseguir los títulos de las series más populares
    popularTV = soup.select("td.titleColumn a[href^='/title']")
    popularTV = [name.text for name in popularTV]
    popularTV = list(map(lambda x:x.strip(),popularTV))
    top5 = popularTV[:6]
    
    #Conseguir el score de las series más populares
    scores = soup.select("td.ratingColumn strong")
    score = [score.text for score in scores]
    top5score = score[:6]
    
    #Crear el dataframe de las series más populares en IMDB
    columns = ["Name", "Score"]
    dicttop = {'Name':top5,'Score':top5score}

    global top5df
    
    top5df = pd.DataFrame(dicttop, columns=columns)
    return top5df

In [3]:
get_top_five()

,Name,Score
0,Watchmen,6.8
1,Cómo vivir contigo mismo,7.4
2,Peaky Blinders,8.8
3,The Walking Dead,8.3
4,Modern Love,8.1
5,American Horror Story,8.1


In [4]:
def access_Twitter_API(): 
    # Mis credenciales 
    API_KEY = "CRuEovkPCCTmYVAC8PbndD9N3"
    API_SECRET = "SikynKdtF2NpEDpx9xvdtuS9wMHbgQXFAyCq5qj1HYbCJfnhEZ"
    ACCESS_TOKEN = "79345899-q0Yb2A2BOfNXkNEjqdFU474r8yARK2WkUqhlTYDjY"
    ACCESS_TOKEN_SECRET = "OUD4HI3o11cc4VC2TkTcxSYWTQAcjsh2mlni3SGlkA3fy"


    # Intento de conexión
    try: 
        auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        global api
        api = tweepy.API(auth)
        return api
    except: 
        print("Error: verifica tus credenciales")

In [5]:
access_Twitter_API()

In [20]:
#Convertir a json dataframe
def get_text(json):
    global texts
    global df
    texts = [pd.Series(i._json) for i in json]
    df = pd.DataFrame(texts)
    return df
       

In [27]:
#Buscar los mejores tweets de una serie
def get_best_tweets(serie):
    try:
        global best_tweets
        best_tweets = api.search(q=serie, lang = 'en', count = 50, result_type='mixed')
        global tweets
        tweets = get_text(best_tweets)
        return tweets
    except tweepy.TweepError as e: 
        print("Error : " + str(e)) 

In [54]:
#Dejar solo texto y convertirlo a dataframe
def only_text_column(column):
    global tweets_to_text
    tweets_to_text = tweets['text']
    global tweetsdf
    tweetsdf = pd.DataFrame(tweets_to_text)
    return tweetsdf


In [55]:
def todo(serie):
    return only_text_column(get_best_tweets(serie))

In [56]:
list1 = todo(top5df.iloc[0]['Name'])
list1


,text
0,Are you ready to venture into the great beyond...
1,It's only just begun. #WatchmenHBO https://t.c...
2,What DC Comics needs to learn from Joker’s suc...
3,RT @Firannion: @neilhimself There needs to be ...
4,RT @watchmen: Are you ready to venture into th...
5,RT @Firannion: @neilhimself There needs to be ...
6,RT @55mmbae: The opening scene of watchmen was...
7,"RT @Salon: ""Watchmen"" creator and star: This i..."
8,Last week's #WatchmenHBO was full of tiny deta...
9,"The Affair Comes to an End, Power Heads for Hi..."


In [ ]:
#Esto no sale :( y no sé por qué porque con list1 sí sale, pero con las otras ya no. 
list1,list2,list3,list4,list5 = top5df['Name'].apply(todo)


In [48]:
#función para clasificar tweets
def get_tweet_sentiment(tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(tweet) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

In [57]:
#Aplicar sentimientos a todo
sentiments = tweetsdf['text'].apply(get_tweet_sentiment)

In [58]:
#Asignar resultado al dataframe
tweetsdf['sentiments'] = sentiments

,text,sentiments
0,Are you ready to venture into the great beyond...,positive
1,It's only just begun. #WatchmenHBO https://t.c...,neutral
2,What DC Comics needs to learn from Joker’s suc...,positive
3,RT @Firannion: @neilhimself There needs to be ...,positive
4,RT @watchmen: Are you ready to venture into th...,positive
5,RT @Firannion: @neilhimself There needs to be ...,positive
6,RT @55mmbae: The opening scene of watchmen was...,neutral
7,"RT @Salon: ""Watchmen"" creator and star: This i...",negative
8,Last week's #WatchmenHBO was full of tiny deta...,negative
9,"The Affair Comes to an End, Power Heads for Hi...",positive


In [77]:
def get_positive_percentages(column):
    global ltweets
    ltweets = len(column)
    
    # picking positive tweets from column 
    global ptweets
    ptweets = [tweet for tweet in column if tweet == 'positive']
    global lptweets
    lptweets = len(ptweets)
    global percentagePos
    percentagePos = 100*len(ptweets)/ltweets
    
    # percentage of positive tweets 
    return("Positive tweets percentage: {} %".format(100*len(ptweets)/ltweets))    
    

In [91]:
def get_negative_percentages(column):
    global lntweets
    ltweets = len(column)
    
    # picking negative tweets from column 
    global ntweets
    ntweets = [tweet for tweet in column if tweet == 'negative'] 
    global lntweets
    lntweets = len(ntweets)
    global percentageNeg
    percentageNeg = 100*len(ntweets)/ltweets
    
    # percentage of negative tweets 
    return("Negative tweets percentage: {} %".format(100*len(ntweets)/ltweets)) 

In [92]:
get_negative_percentages(tweetsdf['sentiments'])

'Negative tweets percentage: 28.0 %'

In [80]:
get_positive_percentages(tweetsdf['sentiments'])

'Positive tweets percentage: 48.0 %'

In [82]:
#La idea es que cada serie tenga el score y al lado el porcentaje de tweets postivos y tweets negativos
#para ver si había una relación, pero no me dio la vida ni el cerebro para hacerlo para cada serie porque mi 
#apply no jaló y ya no quería llorar.

top5df['% tweets positivos'] = percentagePos

In [93]:
top5df['% tweets negativos'] = percentageNeg

In [94]:
#Ejemplo a medias de cómo sería la tabla final.
top5df

,Name,Score,% tweets positivos,% tweets negativos
0,Watchmen,6.8,48.0,28.0
1,Cómo vivir contigo mismo,7.4,48.0,28.0
2,Peaky Blinders,8.8,48.0,28.0
3,The Walking Dead,8.3,48.0,28.0
4,Modern Love,8.1,48.0,28.0
5,American Horror Story,8.1,48.0,28.0
